# Amazon Review Clustering

Miloy Ajmera (majmera@andrew.cmu.edu) / 17-634: Applied Machine Learning, Spring 2023

In [ ]:
import os
import sklearn
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random

import seaborn as sns  # We'll mostly use seaborn, a stats visualization library
import matplotlib.pyplot as plt  # Seaborn wrap Matplotlib for visualization. We'll occasionaly want to make custom changes to the plots.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 amazon
 amazon.model
'Amazon Review Clustering.ipynb'
'Amazon Review Clustering - MetaData.ipynb'
'Amazon Review Clustering - Small Sample.ipynb'
 counts.csv
 list.xlsx


In [ ]:
filepath = "/content/drive/Othercomputers/My PC/Carnegie Mellon University Notes/Semester 3 - Spring 2023/Applied Machine Learning/Homework Assignments/Amazon/"

## Load the Amazon Review dataset.
Read and prepare the data. Consider how you plan to deal with missing data: will you remove columns, if so, under what conditions? Will you attempt to impute any data?

In [ ]:
# get the contents and size of the json file - 2 mins 45 secs
filename = 'amazon/meta_Home_and_Kitchen.json'
# Load the dataframes
num_lines = sum(1 for line in open(filepath + filename))
print(num_lines)

1300540


In [ ]:
# Using random library and examples from https://docs.python.org/3/library/random.html
# Taking random samples - 0 secs
lines_to_be_considered = num_lines / 100000
chunk_size = int(num_lines / 1000)
total_range = range(0, num_lines)

ranges = []
for i in range(0, len(total_range), chunk_size) :
  ranges.append(total_range[i:i+chunk_size])

random_values = []
for split_range in ranges:
  random_values.append(random.sample(split_range, int(len(split_range)/100)))

random_list = [random_value for sublist in random_values for random_value in sublist]

print(len(random_list) * 100 / num_lines)

0.999969243544989


In [ ]:
# get ranges - 0 secs
df_range = range(0, num_lines)
ranges = []
for i in range(0, len(df_range), int(len(df_range)/10)) :
  ranges.append(df_range[i:i+int(len(df_range)/10)])

ranges = ranges[:-1]
print(ranges)

[range(0, 130054), range(130054, 260108), range(260108, 390162), range(390162, 520216), range(520216, 650270), range(650270, 780324), range(780324, 910378), range(910378, 1040432), range(1040432, 1170486)]


In [ ]:
# understand data - 2 mins  54 secs
import csv
import json
json_file = open(filepath + filename, "r")
# for ri, single_range in enumerate(ranges):
data = []
for i, line in enumerate(json_file):
    if i in ranges[0]:
      data.append(json.loads(line))
df = pd.DataFrame.from_records(data)
df.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Home & Kitchen, Kitchen & Dining, Dining & En...",,[It was a time honored tradition among the ear...,,You Are Special Today Red Plate [With Red Pen],"[B0001XR2F2, B01LY51HUN, B07CXZ9C5B, 0310258952]",[],,Waechtersbach USA,[],"[>#39,665 in Kitchen & Dining (See Top 100 in ...","[B0001XR2F2, B00MOFKX1A, B07G3LN13B, B07CYXMFF...",Amazon Home,"class=""a-bordered a-horizontal-stripes a-spa...","October 8, 2006",$37.00,0001487795,NaN
1,"[Home & Kitchen, Home Dcor, Candles & Holders,...",,[VICKS INHALER relieves stuffy noses helps sin...,,Vicks Inhaler Relief for Cold Sinus Nasal Cong...,[],[],,Vicks,[],"[>#1,763,185 in Home & Kitchen (See Top 100 in...",[B00UPCRZEC],Amazon Home,,,$4.05,0002020300,NaN
2,"[Home & Kitchen, Kitchen & Dining, Dining & En...",,"[16 oz squeeze bottle, 1 lb.]",,Artistic Churchware Communion Cup Filler: RW525,[],[],,Artistic Churchware,"[Religious Supply Center, RW-525, Communion Cu...","[>#2,127,003 in Home & Kitchen (See Top 100 in...","[B00C9J79TA, B00XB3ZG5M, B0714K8MSR, 078472530...",Amazon Home,,,$12.48,0006564224,NaN
3,"[Home & Kitchen, Bath, Bathroom Accessories]",,[The only soap in the world with a unique comb...,,4 BARS! Mysore Sandal Soap 70grams FAST SHIPPING,[],[https://images-na.ssl-images-amazon.com/image...,,Mysore,[],"[>#6,942,841 in Home & Kitchen (See Top 100 in...",[B001G7PZB0],Amazon Home,,,$22.00,0009046461,NaN
4,"[Home & Kitchen, Home Dcor, Home Fragrance, In...",,"[Divya Arogya vati improves health, It has nat...",,AROGYA VATI (40gm) by popeye seller,[],[],,Patanjali,[],"[>#3,103,399 in Home & Kitchen (See Top 100 in...","[B075M91MX3, 0258798785, 6565652554]",Amazon Home,,,$5.10,0234937912,NaN


In [ ]:
# take list of all categories from meta data - 3 mins 52 secs
import csv
import json
from collections import Counter
json_file = open(filepath + filename, "r")
data = []
for i, line in enumerate(json_file):
      json_object = json.loads(line)
      data.extend(json_object['category'])
df = pd.DataFrame(data, columns=['category'])
df.head()

,category
0,Home & Kitchen
1,Kitchen & Dining
2,Dining & Entertaining
3,Dinnerware
4,Plates


In [ ]:
from pandas.core.indexes import category
# Take the 50% categories whose counts are more than 50,000 - 1 sec
category_list = df['category'].value_counts()
category_list = category_list.iloc[9:12]
category_list


Storage & Organization    78111
Furniture                 75419
Bath                      53144
Name: category, dtype: int64

In [ ]:
# get the total of values
total = category_list.sum()
print("Percentage of " + str(total) + " categories picked out of " + str(num_lines) + " = " + str(total * 100 / num_lines))
list_of_categories = category_list.index.tolist()
list_of_categories

Percentage of 206674 categories picked out of 1300540 = 15.891398957356175


['Storage & Organization', 'Furniture', 'Bath']

In [ ]:
# Take the asin's for the top 12 categories - 4 mins 0 secs
json_file = open(filepath + filename, "r")
data_list = []
for i, line in enumerate(json_file):
        json_object = json.loads(line)
        list_of_categories = json_object['category']
        data = {}
        for each_category in list_of_categories:
            if each_category in category_list:
              data['asin'] = json_object['asin']
              data['category'] = json_object['category']
              data_list.append(data)
              break
data_list[:10]

[{'asin': '0009046461',
  'category': ['Home & Kitchen', 'Bath', 'Bathroom Accessories']},
 {'asin': '0555667502',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Travel & To-Go Food Containers',
   'Lunch Bags']},
 {'asin': '0938985329',
  'category': ['Home & Kitchen',
   'Storage & Organization',
   'Home Storage Hooks']},
 {'asin': '1581174292',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Food Storage',
   'Cookie Jars']},
 {'asin': '1605534137',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Food Storage',
   'Food Storage & Organization Sets']},
 {'asin': '1616619813',
  'category': ['Home & Kitchen', 'Furniture', "Kids' Furniture"]},
 {'asin': '163375233X',
  'category': ['Home & Kitchen',
   'Kitchen & Dining',
   'Storage & Organization',
   'Travel & To-Go Food Containers',
   'Reusable Grocery Bags']},
 {'asin': '1909558672',
  'category': ['Home & 

In [ ]:
#Get unique list of asins and put them in a list
asins = []
for item in data_list:
    asins.append(item['asin'])
unique_asins = set(asins)
print("Percentage of " + str(len(unique_asins)) + " asins picked out of " + str(num_lines) + " = " + str(len(unique_asins) * 100 / num_lines))


Percentage of 205126 asins picked out of 1300540 = 15.772371476463622


### Use Home and Kitchen JSON file and map the corresponding asins

In [ ]:
# get the contents and size of the json file
filename = 'amazon/Home_and_Kitchen.json'
# Load the dataframes
num_lines = sum(1 for line in open(filepath + filename))
print(num_lines)

21928568


In [ ]:
# understand data - 11 mins
import csv
import json
with open(filepath + 'amazon/asin-data.json', 'w') as writer_file:
  json_file = open(filepath + filename, "r")
  data = []
  for i, line in enumerate(json_file):
    data = json.loads(line)
    if data['asin'] in unique_asins:
      json.dump(data, writer_file)
      writer_file.write('\n')
  
# df = pd.DataFrame.from_records(data)
# df.head()


### Read the saved asins json file

In [ ]:
# get the contents and size of the json file
filename = 'amazon/asin-data.json'
# Load the dataframes
num_asin_lines = sum(1 for line in open(filepath + filename))
print(num_asin_lines)

4681185


In [ ]:
print("Percentage of " + str(num_asin_lines) + " asins picked out of " + str(num_lines) + " = " + str(num_asin_lines * 100 / num_lines))

Percentage of 4681185 asins picked out of 21928568 = 21.347426790477154


In [ ]:
# import random
# random_list = random.sample(range(num_asin_lines), int(num_asin_lines/10))
# random_set = set(random_list)
# print(len(random_set))

In [ ]:
# print("Percentage of " + str(len(random_set)) + " asins picked out of " + str(num_lines) + " = " + str(len(random_set) * 100 / num_lines))

In [ ]:
import json
import pandas as pd
# extract data into a dataframe - 2 mins
json_file = open(filepath + 'amazon/asin-data.json', "r")
data = []
for i, line in enumerate(json_file):
      # if i in random_set:
    data.append(json.loads(line))
df_filtered = pd.DataFrame.from_records(data)
df_filtered.head()


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"08 9, 2013",ACQGF9PXBB35M,1581174292,{'Format:': ' Hardcover'},Ed Mirabal,Its a nice variation on the kids book and the ...,Kids book,1376006400,NaN,NaN
1,5.0,True,"02 2, 2013",A3P565UNR58SPG,1581174292,{'Format:': ' Hardcover'},Monnie D. Brewster,"The book teaches many things, how to count, to...",Very good for kid and we love it too.,1359763200,NaN,NaN
2,5.0,True,"01 12, 2013",A3LT7ZV6LZZF7N,1581174292,{'Format:': ' Hardcover'},tetani,Purchased this book as a gift for the grandchi...,Fun book,1357948800,NaN,NaN
3,5.0,True,"11 29, 2012",A13S4950ZDHDQG,1581174292,{'Format:': ' Hardcover'},Kindle Customer,We read this EVERY day at daycare. Santa has ...,My children's favorite book!,1354147200,NaN,NaN
4,3.0,True,"07 22, 2012",A13QQN9O57I2HO,1581174292,{'Format:': ' Hardcover'},L,This book is cute my 18 month old likes it. I...,Cute.,1342915200,NaN,NaN


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681185 entries, 0 to 4681184
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   style           object 
 6   reviewerName    object 
 7   reviewText      object 
 8   summary         object 
 9   unixReviewTime  int64  
 10  vote            object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 397.3+ MB


### Lets filter the dataset to provide reviews which should have meaningful texts

In [ ]:
df_filtered = df_filtered[(df_filtered['verified'] == True) & (df_filtered['vote'].notna()) & (df_filtered['overall'] > 4.0)]
df_filtered.to_json(filepath + 'amazon/asin-data-filtered.json', orient='records')
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274678 entries, 16 to 4681169
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         274678 non-null  float64
 1   verified        274678 non-null  bool   
 2   reviewTime      274678 non-null  object 
 3   reviewerID      274678 non-null  object 
 4   asin            274678 non-null  object 
 5   style           169770 non-null  object 
 6   reviewerName    274665 non-null  object 
 7   reviewText      274234 non-null  object 
 8   summary         274596 non-null  object 
 9   unixReviewTime  274678 non-null  int64  
 10  vote            274678 non-null  object 
 11  image           44230 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 33.5+ MB


In [ ]:
# extract the dataframe into a new dataframe
df_filtered = pd.read_json(filepath + 'amazon/asin-data-filtered.json', orient='records')
df_filtered = df_filtered[['reviewText']]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274678 entries, 0 to 274677
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewText  274234 non-null  object
dtypes: object(1)
memory usage: 2.1+ MB


### Now perform LDA on the extracted dataset

In [ ]:
from gensim import models, similarities
from gensim.corpora.dictionary import Dictionary
# Plus a few other assorted inputs.
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Clean the data
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: np.nan if len(str(x)) < 2 else x)
df_filtered = df_filtered.dropna()
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: x.lower())
df_filtered.head()

,reviewText
0,great mat for theme or general play. it doesn'...
1,these are just what i was needing to keep my p...
2,i absolutely love these!! they are the perfect...
3,"<div id=""video-block-r10wxkxpgc8kma"" class=""a-..."
4,"for what i needed, these are great little cont..."


In [ ]:
# Removing stop words
import pandas as pd
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: ' '.join([word for word in str(x).split() if str(word) not in (stop_words)]))
df_filtered.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,reviewText
0,great mat theme general play. really keep ship...
1,needing keep pills supplements ready taking mo...
2,absolutely love these!! perfect size many thin...
3,"<div id=""video-block-r10wxkxpgc8kma"" class=""a-..."
4,"needed, great little containers. reviewers say..."


In [ ]:
# Here we start from full sentences again to help the NLP library understand the types of words. 
# Note: this can take quite a while! - 5 mins
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(word) for word in word_tokenize(text)]
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lemmatize_text)
df_filtered.head()

,reviewText
0,"[great, mat, theme, general, play, ., really, ..."
1,"[needing, keep, pill, supplement, ready, takin..."
2,"[absolutely, love, these, !, !, perfect, size,..."
3,"[<, div, id=, '', video-block-r10wxkxpgc8kma, ..."
4,"[needed, ,, great, little, container, ., revie..."


In [ ]:
# We'd typically start by tokenizing the data 
from gensim.utils import tokenize
def tokenize_text(text):
    return list(tokenize(text))
df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: tokenize_text(x))
df_filtered.head()


In [ ]:
# And then stem all words.
from gensim.parsing.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_text(word):
    return stemmer.stem(word)

df_filtered['reviewText'] = df_filtered['reviewText'].apply(lambda x: [stemmer.stem(word) for word in x])
df_filtered.head()

In [ ]:
# Create a corpus from a list of texts.
# The dictionary just extracts and numbers each distinct word.
dictionary = Dictionary(df_filtered['reviewText'])
# A corpus is a sparse datastore containing the number of times each word appears in each document.
corpus = [dictionary.doc2bow(text) for text in df_filtered['reviewText']]


In [ ]:
# Print a sample of dictionary items.
top_words_in_doc_0 = sorted(corpus[0], key=lambda e: e[1], reverse=True)[:10]
for word_index, count in top_words_in_doc_0:
  print(f'{dictionary[word_index]}   \t\t index: {word_index}\tcount: {count:,}')

.   		 index: 1	count: 3
ship   		 index: 19	count: 2
,   		 index: 0	count: 1
amazing   		 index: 2	count: 1
anything   		 index: 3	count: 1
backdrop   		 index: 4	count: 1
easier   		 index: 5	count: 1
everyday   		 index: 6	count: 1
game   		 index: 7	count: 1
general   		 index: 8	count: 1


In [ ]:
# Let's start with a simple TF-IDF model. Worth noting: SKLearn also has a Tfidf vectorizer.
model = models.TfidfModel(corpus)

In [ ]:
# What are the most _salient_ words in document 0?
top_tfidf_words = sorted(model[corpus[0]], key=lambda e: e[1], reverse=True)[:10]
for word_index, score in top_tfidf_words:
  print(f'{dictionary[word_index]}   \t\t index: {word_index} \t\t tf-idf: {score:.2f}')

# What's different from the previous ranking? What do you think the topic label might be?

x-wing   		 index: 25 		 tf-idf: 0.46
ship   		 index: 19 		 tf-idf: 0.40
backdrop   		 index: 4 		 tf-idf: 0.30
mouse   		 index: 15 		 tf-idf: 0.23
general   		 index: 8 		 tf-idf: 0.23
identical   		 index: 10 		 tf-idf: 0.23
theme   		 index: 24 		 tf-idf: 0.22
everyday   		 index: 6 		 tf-idf: 0.19
game   		 index: 7 		 tf-idf: 0.19
sliding   		 index: 22 		 tf-idf: 0.19


In [ ]:
# Okay, let's do something more exciting: build an LDA model.
# Note: we could also do this with SKLearn using LinearDiscriminantAnalysis. - 5m 46s

# Let's start with 20 topics.
num_topics = 20
model = models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=2, workers=4, dtype=np.float64)

In [ ]:
# Let's again look at the most salient words per topic, but note that we no longer
# have "labels" now.
for ix in range(num_topics):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')  # See any patterns?

0:  organized makeup it organizer one drawer love . , !
1:  work like mat sink water soap one shower , .
2:  cushion couch sit leg seat back comfortable , chair .
3:  hold laundry ! great clothes use one bag , .
4:  one rack closet top hook wall screw hanger , .
5:  frame perfect ... , great . it love bed !
6:  , ? . class= ; `` < > & ''
7:  one comfortable i like sleep foam bed mattress , .
8:  put kitchen great fit space rack look hold shelf .
9:  took would instruction desk one piece put together , .
10:  look , size chair perfect it great ! love .
11:  day month 've time one still i year . ,
12:  side inch '' shelf one top tv stand , .
13:  paper work well one towel product holder , * .
14:  ... gift 've one 's it 'm , i .
15:  i `` '' -- . - : , ( )
16:  cabinet storage small one customer box service door , .
17:  look .. quality , bathroom shower towel curtain color .
18:  keep box one use food lid lunch container , .
19:  put sturdy quality well great look easy table , .


In [ ]:
# How coherent are these topics?
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model, texts=df_filtered['reviewText'], dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
coherence

0.4157494109924694